In [3]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
import pandas as pd
train = pd.read_csv('Loan Prediction/train.csv', index_col=0)
train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [41]:
train.shape

(614, 12)

In [42]:
train.dtypes

Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [43]:
train.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [44]:
X, y = train.drop('Loan_Status', axis=1), train['Loan_Status']

In [45]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

simp_imp_object = SimpleImputer(strategy='constant', fill_value='unknown' )
simp_imp_numeric = SimpleImputer(strategy='median')

from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

ct1 = make_column_transformer(
    (simp_imp_object, make_column_selector(dtype_include=object)),
    (simp_imp_numeric, make_column_selector(dtype_exclude=object)),
    verbose_feature_names_out=False
).set_output(transform='pandas')

ct2 = make_column_transformer(
    (ohe, make_column_selector(dtype_include=object)),
    verbose_feature_names_out=False
).set_output(transform='pandas')


In [46]:
X = ct1.fit_transform(X)
X = ct2.fit_transform(X)

In [47]:
X

,Gender_Male,Gender_unknown,Married_Yes,Married_unknown,Dependents_1,Dependents_2,Dependents_3+,Dependents_unknown,Education_Not Graduate,Self_Employed_Yes,Self_Employed_unknown,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,
LP001002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
LP001003,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LP001005,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
LP001006,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
LP001008,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
LP002978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LP002979,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
LP002983,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [54]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import numpy as np

lr = LogisticRegression()
l1_ratio = np.linspace(0.001, 1, 5)

scl_std, scl_mm = StandardScaler(), MinMaxScaler()


params = {'LR__penalty': ['l1', None, 'l2', 'elasticNet'],
          'LR__multi_class': ['ovr', 'multinomial'],
          'LR__l1_ratio': l1_ratio,
          'LR__solver': ['lbfgs', 'sag', 'saga', 'liblinear', 'newton-cg', 'newton-cholesky'],
         'SCL' : [None, scl_std, scl_mm]}

pipe = Pipeline(
    [
        ('SCL', None),
        ('LR', lr)
    ]
    
)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
gcv_lr = GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='accuracy')
gcv_lr.fit(X, y)
print(gcv_lr.best_params_)
print(gcv_lr.best_score_)

{'LR__l1_ratio': 0.001, 'LR__multi_class': 'ovr', 'LR__penalty': 'l1', 'LR__solver': 'saga', 'SCL': None}
0.682380381180861


In [60]:
test = pd.read_csv('Loan Prediction/test.csv', index_col=0)
test.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_ID,,,,,,,,,,,
LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban


In [49]:
test.shape

(367, 12)

In [61]:
test = ct1.transform(test)
test = ct2.transform(test)
y_pred = gcv_lr.predict(test)

In [64]:
submit = pd.read_csv('Loan Prediction/sample_submission.csv')
submit['Loan_Status'] = y_pred

In [66]:
submit.to_csv('submit.csv', index=False)